<img src="https://raw.githubusercontent.com/brazil-data-cube/code-gallery/master/img/logo-bdc.png" align="right" width="64" />

# <span style="color:#336699">Image processing on images obtained through STAC</span>
<hr style="border:2px solid #0077b9;">

<div style="text-align: left;">
    <a href="https://nbviewer.jupyter.org/github/brazil-data-cube/code-gallery/blob/master/jupyter/Python/stac/stac-image-processing.ipynb"><img src="https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg" align="center"/></a>
</div>

<br/>

<div style="text-align: center;font-size: 90%;">
    Rennan Marujo<sup><a href="https://orcid.org/0000-0002-0082-9498"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Gilberto R. Queiroz<sup><a href="https://orcid.org/0000-0001-7534-0219"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Felipe Menino Carlos<sup><a href="https://orcid.org/0000-0002-3334-4315"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>
    <br/><br/>
    Earth Observation and Geoinformatics Division, National Institute for Space Research (INPE)
    <br/>
    Avenida dos Astronautas, 1758, Jardim da Granja, São José dos Campos, SP 12227-010, Brazil
    <br/><br/>
    Contact: <a href="mailto:brazildatacube@inpe.br">brazildatacube@inpe.br</a>
    <br/><br/>
    Last Update: December 08, 2021
</div>

<br/>

<div style="text-align: justify;  margin-left: 15%; margin-right: 15%;">
<b>Abstract.</b> This Jupyter Notebook describes how to search for CBERS-4 data products in <em>Brazil Data Cube</em>'s catalog through the STAC service. Then it shows how to use Python libraries to perform some image processing. It starts by computing the Normalized Difference Vegetation Index (NDVI) based on the red and near-infrared spectral bands. Next, it demonstrates a threshold analysis based on the computed NDVI. Lastly, it computes the NDVI difference for images from two diffrent dates.
</div>

<br/>
<div style="text-align: justify;  margin-left: 25%; margin-right: 25%;font-size: 75%; border-style: solid; border-color: #0077b9; border-width: 1px; padding: 5px;">
    <b>For an introduction to the SpatioTemporal Asset Catalog (STAC) with the <em>Brazil Data Cube</em> infrastructure, please, refer to the following Jupyter Notebook:</b>
    <div style="margin-left: 10px; margin-right: 10px">
    Zaglia, M.; Marujo, R.; Queiroz, G. R.; Carlos, F. M. <a href="./stac-introduction.ipynb" target="_blank">Introduction to the SpatioTemporal Asset Catalog (STAC)</a>.
    </div>
</div>


<img src="https://raw.githubusercontent.com/brazil-data-cube/code-gallery/master/img/stac/stac.png?raw=true" align="right" width="300"/>

# STAC Client API
<hr style="border:1px solid #0077b9;">

For running the examples in this Jupyter Notebook you will need to install the [STAC client for Python](https://github.com/brazil-data-cube/stac.py). To install it from PyPI using `pip`, use the following command:

In [ ]:
!pip install pystac-client rasterio shapely matplotlib

In order to access the funcionalities of the client API, you should import the `pystac_client` package, as follows:

In [ ]:
import pystac_client

After that, you can check the installed `pystac_client` package version:

In [ ]:
pystac_client.__version__

Then, create a `pystac_client.Client` object attached to the Brazil Data Cube' STAC service:

In [ ]:
parameters = dict(access_token='change-me')
service = pystac_client.Client.open('https://brazildatacube.dpi.inpe.br/stac/', parameters=parameters)

# Searching for CBERS-4 Images
<hr style="border:1px solid #0077b9;">

We are going to use the STAC `search` API to look for images from the data cube named `CB4_64_16D_STK`. This data cube is a temporal composite from CBERS-4/AWFI surface reflectance data. Let's define a search box with the following bounds: $x_{min} = -45.9$, $x_{max} = -45.4$, $y_{min} = -12.9$, $y_{max} = -12.6$. Besides that, the period of interest ranges from August 1st, 2018 to July 31st, 2019.

In [ ]:
bbox = (-45.9, -12.9, -45.4, -12.6)

In [ ]:
item_search = service.search(collections=['CB4_64_16D_STK-1'],
                             bbox=bbox,
                             datetime='2018-08-01/2019-07-31')

In [ ]:
item_search

The above query, should return 24 items for this particular data cube:

In [ ]:
item_search.matched()

# Spectral Indices
<hr style="border:1px solid #0077b9;">

Spectral indices are computed using sensor bands to highlight a certain feature of a target or reduce certain effects. In this context, vegetation indices are spectral indices that enhance characteristics of vegetation, using bands such as Near Infra-red (`NIR`), a region where vegetation shows the most intense reflectance and bands located in red, where the vegetation has the highest absorption of visible sunlight due to the presence in its constitution of the green pigment chlorophyll (Meneses, 2012). The behavior of these spectral bands in some types of targets can be observed below:

<center>
<img src="https://brazil-data-cube.github.io/_images/spectral-profile.png" width="480" />
<br/>
Spectral profile of several targets. Source: modified from Pan et. al (2015).
</center>

# Calculating the Normalized Difference Vegetation Index (NDVI)
<hr style="border:1px solid #0077b9;">

The normalized difference vegetation index (NDVI) is calculated using the **Red** and **Near Infrared** (NIR) spectral bands. It assesses whether or not the target being observed contains live green vegetation. It can be calculated through the following equation:

$$
NDVI = \frac{(NIR - RED)}{(NIR + RED)}
$$

<center><b>Equation 1</b> - NDVI.</center>

We are going to compute the NDVI just with images from the first item:

In [ ]:
items = list(item_search.get_items())

In [ ]:
item = items[0]
item

In order to know which band is available in an item, one can access its metadata:

In [ ]:
sorted(item.properties['eo:bands'], key=lambda band: band['name'])

<div style="text-align: justify;  margin-left: 15%; margin-right: 15%; border-style: solid; border-color: #0077b9; border-width: 1px; padding: 5px;">
    <b>Note:</b> As one can see, in the above metadata the Brazil Data Cube already provides the <em>NDVI</em> and <em>EVI</em> alongside with the spectral bands for the data cubes, besides the quality indicators (<em>CLEAROB</em>, <em>PROVENANCE</em>, <em>CMASK</em>, <em>TOTALOB</em>).
</div>

As one can see in the above cell, the `BAND15` correspond to the red wavelength and `BAND16` to the near-infrared.

We have implemented a basic `read` method that allows to retrieve part of an image according to a rectangle specified in `EPSG:4326`. The next cells reads the `red` and `nir` bands:

In [ ]:
import rasterio
from rasterio.crs import CRS
from rasterio.warp import transform
from rasterio.windows import from_bounds

def read(uri: str, bbox: list, masked: bool = True, crs: str = None):
    """Read raster window as numpy.ma.masked_array."""
    source_crs = CRS.from_string('EPSG:4326')
    if crs:
        source_crs = CRS.from_string(crs)

    # Expects the bounding box has 4 values
    w, s, e, n = bbox
        
    with rasterio.open(uri) as dataset:
        transformer = transform(source_crs, dataset.crs, [w, e], [s, n])
        window = from_bounds(transformer[0][0], transformer[1][0], 
                             transformer[0][1], transformer[1][1], dataset.transform)
        return dataset.read(1, window=window, masked=masked)

In [ ]:
red = read(item.assets['BAND15'].href, bbox=bbox)

<div style="text-align: justify;  margin-left: 15%; margin-right: 15%; border-style: solid; border-color: #0077b9; border-width: 1px; padding: 5px;">
    <b>Note:</b> If there are errors because of your pyproj version, you can run the code below as specified in <a  href="https://rasterio.readthedocs.io/en/latest/faq.html#why-can-t-rasterio-find-proj-db-rasterio-from-pypi-versions-1-2-0" target="_blank">rasterio documentation</a> and try again:

       import os
       del os.environ['PROJ_LIB']
</div>

In [ ]:
red

In [ ]:
nir = read(item.assets['BAND16'].href, bbox=bbox)

In [ ]:
nir

Let's take a look at the retrieved data:

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.imshow(red, cmap='gray');

In [ ]:
plt.imshow(nir, cmap='gray');

Finally, let's compute the NDVI:

In [ ]:
ndvi = (nir - red)/(nir + red)
ndvi

Use the Matplotlib to visualize the result array:

In [ ]:
plt.imshow(ndvi, cmap='gray');

We can also plot using other colormaps:

In [ ]:
plt.imshow(ndvi, cmap='jet');

For more colormaps check https://matplotlib.org/stable/tutorials/colors/colormaps.html

# Histogram
<hr style="border:1px solid #0077b9;">

The histogram of a digital image, also known as frequency distribution, is the graphic representation in columns showing the intensity of values and the number of pixels with such intensity and is the basis for several types of digital image processing (Gonzalez & Woods, 2007). Some types of histogram can be observed below:

<center>
<img src="https://brazil-data-cube.github.io/_images/histogram.png" width="480" />
<br/>
Four types of images: dark, bright, low contrast and high contrast, and their respective histograms. Source: (Gonzalez & Woods, 2007).
</center>

# Thresholding images
<hr style="border:1px solid #0077b9;">

Digital image Classification is a broad topic and basically consists of assigning labels to targets in a dataset, also called classes. Although many of the classification approaches are complex, there are some simple approaches to solving certain problems, such as thresholding.

The idea of assigning a class following a threshold assumes that the data can be separated by a simple "line", this can be seen in the next figures in a histogram:


<center>
<img src="https://scikit-image.org/docs/dev/_images/sphx_glr_plot_multiotsu_001.png" width="960" />
<br/>
Histogram limiarization example. Source: <a href="https://scikit-image.org/docs/dev/auto_examples/segmentation/plot_multiotsu.html">scikit-image doc</a>
</center>

Let's try to separate our data into groups according to their NDVI values.
But first let's see how the image histogram behaves:

In [ ]:
plt.title("NDVI Histogram")
plt.hist(ndvi)
plt.show()

Supposing we can separate the `ndvi` image with threshold, we would assume for this specific case that:
* all pixels with values below 0.2 are dark pixels;
* all pixels above 0.45 are areas containing a good portion of vegetation.
* all pixels with values from 0.2 to 0.45 are areas with few vegetation;

We can perform this thresholding by selecting in the ndvi matrix all values belonging to a given range and assigning a common integer value. We assume the following integer values:
* `1`: dark pixels;
* `2`: vigorous vegetation;
* `3`: weak vegetation.

Let's first create a copy of the original ndvi matrix:

In [ ]:
labeled_img = ndvi.copy()

Now, we can use the new copy of the ndvi array and assign the values according to each range of values:

In [ ]:
labeled_img[ndvi < 0.2] = 1 # < 0.2
labeled_img[ndvi >= 0.2] = 3 # 0.2 - 0.45
labeled_img[ndvi >= 0.45] = 2 # >= 0.45
labeled_img

Finally, let's see the `ndvi` image separated into those labels:

In [ ]:
plt.rcParams['figure.figsize'] = [30, 20] #Change plot size
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.imshow(ndvi, cmap='gray')
ax2.imshow(labeled_img, cmap='brg');

# Calculating Image Difference
<hr style="border:1px solid #0077b9;">

Now let's suppose we want to compare the NDVI for images from two different dates and same location, for instance to verify the areas where crops have grown and areas that loss vegetation.

For this computation we are going to use the NDVI bands provided in the data cube, and we will select two items (with the same location but with different dates) using STAC.

The first image comprises pixels from July 28th, 2019 to August 12th, 2019 (`2019-07-28_2019-08-12`):

In [ ]:
first_item = item

In [ ]:
first_item

In [ ]:
ndvi_first_image = read(first_item.assets['NDVI'].href, bbox=bbox)

The other selected image comprises pixels from January 01st, 2019 to January 16th, 2019 (`2019-01-01_2019-01-16`) - i.e., six months before the first one selected:

In [ ]:
second_item = items[13]

In [ ]:
second_item

In [ ]:
ndvi_second_image = read(second_item.assets['NDVI'].href, bbox=bbox)

<div style="text-align: justify;  margin-left: 15%; margin-right: 15%; border-style: solid; border-color: #0077b9; border-width: 1px; padding: 5px;">
    <b>Note:</b> NDVI bands precomputed by BDC ranges from <em>-10000</em> to <em>10000</em>, instead of <em>-1</em> to <em>1</em>, as can be seen in the item metadata. This is due to the lower volume required to store files that use 16-bit integer values rather than 32-bit float.
</div>

Considering that these images are from an agricultural area and that crops are normally planted near August (first observation), in a six months, previous or after the first observation, it is expected to find crops, which will imply in greater NDVI values (more vigorous vegetation). This will cause NDVI band to present brighter values on these areas. Using the `gray` colormap, high value NDVI pixels will be more similar to white, while low value NDVI pixels will be closer to the black color.
Based on that, let's visually compare both NDVI images:

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.imshow(ndvi_first_image, cmap='gray')
ax2.imshow(ndvi_second_image, cmap='gray')

Assuming we want to see what have grown and what was loss, let's subtract the most recent image from the oldest one and plot it:

In [ ]:
ndvi_diff = ndvi_first_image - ndvi_second_image

In [ ]:
plt.rcParams['figure.figsize'] = [10, 5]
plt.imshow(ndvi_diff, cmap='jet');

As can be seen in the the NDVI difference plot, the main changes on pixel values were found in agriculture areas, which was expected due to changes in crops.
The blue values indicate negative values, while red values are positive. This means that for the blue areas there was a loss of vegetation, as a decreasing result on the NDVI value, meaning that crops were harvest. Meanwhile, on the red areas, the NDVI value increased as a result of the more vigorous vegetation on the recent date.

# References
<hr style="border:1px solid #0077b9;">

- [Python Client Library for STAC Service](https://pystac-client.readthedocs.io/en/latest/)

- [Spatio Temporal Asset Catalog Specification](https://stacspec.org/)

# See also the following Jupyter Notebooks
<hr style="border:1px solid #0077b9;">

* [Introduction to the SpatioTemporal Asset Catalog (STAC)](./stac-introduction.ipynb)